In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd My\ Drive/cotton-market-trend-prediction

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
/gdrive/My Drive/cotton-market-trend-prediction


In [4]:
import numpy as np
import pandas as pd
from pandas import datetime
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
from matplotlib import pyplot as plt
import time

pd.set_option('display.max_rows',50)

Using TensorFlow backend.


In [5]:
df = pd.read_csv('gujarat_market_data.csv')
df['district'] = df['district'].astype(str)
df['market'] = df['market'].astype(str)
df['commodity'] = df['commodity'].astype(str)
df['variety'] = df['variety'].astype(str)
df['min_price'] = df['min_price'].astype(float)
df['max_price'] = df['max_price'].astype(float)
df['modal_price'] = df['modal_price'].astype(float)
df['date']= pd.to_datetime(df['date'])
df.info() 
df

market_list=df['market'].unique()
market_list = market_list[market_list != 'Deesa']
market_list = market_list[market_list != 'Damnagar']
market_list = market_list[market_list != 'Kosamba(Vankal)']
market_list = market_list[market_list != 'Kosamba(Zangvav)']
market_list = market_list[market_list != 'Lalpur']
market_list = market_list[market_list != 'Malpur']

market_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76226 entries, 0 to 76225
Data columns (total 9 columns):
si             76226 non-null int64
district       76226 non-null object
market         76226 non-null object
commodity      76226 non-null object
variety        76226 non-null object
min_price      76226 non-null float64
max_price      76226 non-null float64
modal_price    76226 non-null float64
date           76226 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 5.2+ MB


array(['Amirgadh', 'Amod', 'Amreli', 'Anjar', 'Babra', 'Bhavnagar',
       'Bagasara', 'Balasinor', 'Bayad', 'Becharaji', 'Bhiloda',
       'Bodeli(Hadod)', 'Bodeli(Kalediya)', 'Bodeli(Modasar)', 'Botad',
       'Chotila', 'Dhandhuka', 'Dhari', 'Dhoraji', 'Dhrol', 'Gadada',
       'Gogamba', 'Halvad', 'Harij', 'Himatnagar', 'Idar', 'Jam Jodhpur',
       'Jamkandorna', 'Jamnagar', 'Jasdan', 'Jepur Pavi(Chackak)',
       'Jetpur(Dist.Rajkot)', 'Jetpur-Pavi', 'Jhagadiya', 'Junagadh',
       'Kadi', 'Kadi(Kadi cotton Yard)', 'Kalawad',
       'Kapadanj(Moti Jaher)', 'Kapadvanj', 'Karjan', 'Khedbrahma',
       'Kodinar', 'Limdi', 'Mahuva', 'Manavdar', 'Mansa',
       'Mansa(Manas Veg Yard)', 'Meghraj', 'Modasa', 'Modasa(Tintoi)',
       'Morbi', 'Muli', 'Nasvadi', 'Nasvadi(Thalkala)', 'Nizar',
       'Palitana', 'Patan', 'Rajkot', 'Rajpipla', 'Rajula', 'Ranpur',
       'Savarkundla', 'Savli', 'Talod', 'Una', 'Unava', 'Vadhvan',
       'Vagodiya', 'Visavadar', 'Visnagar'], dtype=object)

In [6]:
market_df = {}

for markets in market_list:
  market_df[markets]=df[df.market == markets]
  market_df[markets].to_csv(str(markets)+"_market_data.csv")
  market_df[markets].sort_values(["date"], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:


#create a dictionary of dataframes, one for each cluster
price_df = { markets: pd.DataFrame(pd.read_csv(str(markets)+"_market_data.csv", index_col = ['date'] , parse_dates=['date'])) for markets in market_list }

In [0]:
for markets in market_list:
  price_df[markets]=price_df[markets].filter(["min_price","max_price","modal_price"])
  

In [0]:
data = { markets: pd.DataFrame(price_df[markets].filter(['modal_price'])) for markets in market_list }
dataset = { markets: pd.DataFrame(data[markets].values) for markets in market_list }
training_data_len = {}
for markets in market_list:
  training_data_len[markets] = math.ceil( len(dataset[markets]) * .8 )


In [11]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = {}
for markets in market_list:
  scaled_data[markets] = np.array(scaler.fit_transform(dataset[markets]))
  df2 = df.assign(scaled_data = scaled_data[markets])
  df2.to_csv(str(markets)+"_market_data.csv")


ValueError: ignored

In [0]:
#Create the training data set
#Create the scaled training data set
train_data = {}
x_train = {}
y_train = {}

for markets in market_list:
  sd = scaled_data[markets]
  train_data[markets] = sd[0:training_data_len[markets] , :]
  #Split the data into x_train and y_train data sets
  x_train[markets] = []
  y_train[markets] = []

  for i in range(60, len(train_data[markets])):
    time.sleep(.001)
    x_train[markets].append(np.asarray(train_data[markets][i-60:i, 0]))
    y_train[markets].append(np.asarray(train_data[markets][i, 0]))
    if i<=60:
      print(x_train[markets])
      print(y_train[markets])


In [0]:
#Convert the x_train and y_train to numpy arrays
for x in market_list: 
  x_train[x] = np.array(x_train[x]),
  y_train[x] = np.array(y_train[x])

In [0]:
#Reshape the data
for markets in market_list:
  print(markets)
  train=np.array(x_train[markets])
  print(train.shape)
  x_train[markets] = np.reshape(train, (train.shape[1], train.shape[2], 1))
  print(x_train[markets].shape)
  np.asarray(x_train[markets]).shape

In [0]:
#Build the LSTM model
model={}
for markets in market_list:
  model[markets] = Sequential()
  model[markets].add(LSTM(50, return_sequences=True, input_shape= (x_train[markets].shape[1], 1)))
  model[markets].add(LSTM(50, return_sequences= False))
  model[markets].add(Dense(25))
  model[markets].add(Dense(1))

In [0]:
x_test={}
y_test={}
test_data={}
for models in market_list:
  ds = dataset[models]
  print(type(ds))
  model[models].compile(optimizer='adam', loss='mean_squared_error')
  model[models].fit(x_train[models], y_train[models], batch_size=1, epochs=1)
  #Create the testing data set
  #Create a new array containing scaled values from index 1543 to 2002
  length=int(training_data_len[models]) -60
  print(length)
  sdata = scaled_data[models]
  test_data[models] = sdata[int(length): , :]
  #Create the data sets x_test and y_test
  x_test[models] = []
  
  lenTrain=length+60
  y_test[models] = ds.iloc[lenTrain:, :]
  for i in range(60, len(test_data[models])):
    x_test[models].append(test_data[models][i-60:i, 0])  


In [0]:
for markets in market_list:
  model_json = model[markets].to_json()
  with open(str(markets)+"_model.json", "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model[markets].save_weights(str(markets)+"model.h5")
  print("Saved model to disk :"+str(markets))

In [0]:
#Convert the data to a numpy array
from keras.models import model_from_json
city="Morbi"
x=dataset[city]
X=x.iloc[:,0:]
y=dataset[city]
Y=y.iloc[: ,]
json_file=open(str(city)+"_model.json",'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(str(city)+"model.h5")
print(str(city)+" Model loaded")
loaded_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
score=loaded_model.evaluate(X,Y, verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [0]:
#Get the models predicted price values 
predictions = model[city].predict(x_test_temp)
predictions = scaler.inverse_transform(predictions)